<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">


# Web Scraping for Indeed.com and Predicting Salaries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

### Creating dataframe/reading in csv


In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm

In [3]:
df_lon = pd.read_csv('/Users/harmeetgill/Documents/Jobs/General_assembly/Github_repos/DSI15-lessons/project/project-4/df_london.csv', index_col=0)
df_non = pd.read_csv('/Users/harmeetgill/Documents/Jobs/General_assembly/Github_repos/DSI15-lessons/project/project-4/df_non_london_v2.csv', index_col=0)

In [4]:
concat_df=pd.concat([df_non, df_lon])
concat_df.shape

(6563, 4)

In [5]:
concat_df.head()

,Title,Company,Salary,Location
0,Computer Vision and Machine Learning Engineer,FLOX Ltd,"£30,000 a year",London
1,Machine Learning Scientist - Data Science & AI...,Asos.com,NaN,London
2,Graduate Program - Associate Data Scientist\nnew,RELX Group,NaN,London
3,Data and Research Analyst,Cancer Research UK,"£31,000 a year",Stratford
4,Junior NLP/Machine Learning Developer\nnew,ProWritingAid,NaN,London


## General data cleaning

In [6]:
#removed duplicates
concat_df.drop_duplicates(inplace=True)
concat_df.dropna(axis = 0, inplace=True)

## Salary cleaning 


In [7]:
#removing symbols/characters from salary to leave the integer.

concat_df['Salary'] = concat_df['Salary'].str.replace(',', '')
concat_df['Salary'] = concat_df['Salary'].str.replace('£', '')
concat_df['Salary'] = concat_df['Salary'].str.replace('-', '')

#split up words in salaries column 
split_salary=concat_df["Salary"].str.split(" ", expand = True)

#saving split up data to new cols
concat_df['Min Salary'] = split_salary[0]
concat_df['Max Salary'] = split_salary[2]
concat_df['Salary Period']=split_salary[4]

In [8]:
#made a new column to define whether the salary amount is annual, monthly etc.
concat_df['Salary_freq'] = concat_df.Salary.apply(lambda x: 'year' if 'year' in str(x) 
                                      else 'month' if 'month' in str(x) 
                                      else 'week' if 'week' in str(x)
                                      else 'day' if 'day' in str(x) 
                                      else 'hour' if 'hour' in str(x) 

                                     else None)

In [9]:
#replaced 'year' in max with min salary value
concat_df['Max Salary'] = np.where(concat_df['Max Salary'] == 'year', concat_df['Min Salary'], concat_df['Max Salary'])
concat_df['Max Salary'] = np.where(concat_df['Max Salary'] == 'month', concat_df['Min Salary'], concat_df['Max Salary'])
concat_df['Max Salary'] = np.where(concat_df['Max Salary'] == 'week', concat_df['Min Salary'], concat_df['Max Salary'])
concat_df['Max Salary'] = np.where(concat_df['Max Salary'] == 'day', concat_df['Min Salary'], concat_df['Max Salary'])
concat_df['Max Salary'] = np.where(concat_df['Max Salary'] == 'hour', concat_df['Min Salary'], concat_df['Max Salary'])

#converted salaries to floats
concat_df['Min Salary'] = concat_df['Min Salary'].astype(float)
concat_df['Max Salary'] = concat_df['Max Salary'].astype(float)

In [10]:
#selected, averaged min/max salary and added to new col.
avg_sal=concat_df.iloc[:,4:6]
concat_df['salary_mean'] = avg_sal.mean(axis=1)

In [11]:
concat_df['Salary_freq'].value_counts()

year     971
hour     211
day       44
week      15
month      9
Name: Salary_freq, dtype: int64

In [12]:
#defined a function to normalise all salary periods to annual.
def salary_convert(row):
        if row['Salary_freq'] == 'month' :
            return (row['salary_mean'] * 12)
        elif row['Salary_freq'] == 'week' :
            return row['salary_mean'] * 52
        elif row['Salary_freq'] == 'day' :
            return row['salary_mean'] *256
        elif row['Salary_freq'] == 'hour' :
            return row['salary_mean'] *1538
        elif row['Salary_freq'] == 'year' :
            return row['salary_mean']


In [13]:
concat_df['salary_mean_normalised']=concat_df.apply(lambda row: salary_convert(row), axis=1, result_type='expand')

In [14]:
#dropped NAs in original salary column
concat_df.drop(columns = ['Salary',
                          'Salary Period',
                          'Min Salary',
                          'Max Salary'
                         ], axis=1, inplace=True)


In [15]:
concat_df.shape

(1250, 6)

In [16]:
#removed postcodes
clean_loc = concat_df["Location"].str.split(" ", expand = True)
concat_df['Location'] = clean_loc[0]

In [17]:
concat_df.dropna(axis = 0, inplace=True)

In [18]:
concat_df.shape

(1250, 6)

### Setting salary categories (high/low)

In [19]:
def salary_cutoff (x):
    if x['salary_mean_normalised'] <= 28000 :
        return '0'
    else:
        return '1'

In [20]:
concat_df['High or low (50%)'] = concat_df.apply (lambda x: salary_cutoff(x), axis=1)

In [21]:
concat_df['High or low (75%)'] = concat_df.apply (lambda x: salary_cutoff(x), axis=1)


In [22]:
concat_df.head()

,Title,Company,Location,Salary_freq,salary_mean,salary_mean_normalised,High or low (50%),High or low (75%)
0,Computer Vision and Machine Learning Engineer,FLOX Ltd,London,year,30000.0,30000.0,1,1
3,Data and Research Analyst,Cancer Research UK,Stratford,year,31000.0,31000.0,1,1
5,DATA SCIENTIST | VERV | LONDON OR SHEFFIELD,Austin Consultants,London,year,60000.0,60000.0,1,1
6,Data Scientist - Machine Learning\nnew,Harnham,London,day,525.0,134400.0,1,1
7,Principal Data Scientist\nnew,Nesta,London,year,51500.0,51500.0,1,1


In [23]:
df_to_clean= concat_df

In [24]:
df_to_clean.head()

,Title,Company,Location,Salary_freq,salary_mean,salary_mean_normalised,High or low (50%),High or low (75%)
0,Computer Vision and Machine Learning Engineer,FLOX Ltd,London,year,30000.0,30000.0,1,1
3,Data and Research Analyst,Cancer Research UK,Stratford,year,31000.0,31000.0,1,1
5,DATA SCIENTIST | VERV | LONDON OR SHEFFIELD,Austin Consultants,London,year,60000.0,60000.0,1,1
6,Data Scientist - Machine Learning\nnew,Harnham,London,day,525.0,134400.0,1,1
7,Principal Data Scientist\nnew,Nesta,London,year,51500.0,51500.0,1,1


## Title/Company cleaning

In [25]:
import re
import string

In [26]:
#clean up of strings

def clean_up(x):
#        return x.replace({r'\w+': '')
#     return x.lower()
    x= x.replace('&',' ')
    x=x.replace('\nnew',' ')
    x=x.replace('-',' ')
    x=x.replace('/',' ')
    x=x.replace('|',' ')
    x=x.replace('.',' ')
    x=x.replace('(',' ')
    x=x.replace(')',' ')
    x=x.lower()
    return x

In [27]:
#unclean df title and company is cleaned and saved to new col name
df_to_clean['Title clean']=df_to_clean['Title'].apply(clean_up)

In [28]:
df_to_clean['Company_clean']=df_to_clean['Company'].apply(clean_up)

In [29]:
df_to_clean

,Title,Company,Location,Salary_freq,salary_mean,salary_mean_normalised,High or low (50%),High or low (75%),Title clean,Company_clean
0,Computer Vision and Machine Learning Engineer,FLOX Ltd,London,year,30000.00,30000.00,1,1,computer vision and machine learning engineer,flox ltd
3,Data and Research Analyst,Cancer Research UK,Stratford,year,31000.00,31000.00,1,1,data and research analyst,cancer research uk
5,DATA SCIENTIST | VERV | LONDON OR SHEFFIELD,Austin Consultants,London,year,60000.00,60000.00,1,1,data scientist verv london or sheffield,austin consultants
6,Data Scientist - Machine Learning\nnew,Harnham,London,day,525.00,134400.00,1,1,data scientist machine learning,harnham
7,Principal Data Scientist\nnew,Nesta,London,year,51500.00,51500.00,1,1,principal data scientist,nesta
...,...,...,...,...,...,...,...,...,...,...
4960,"Customer Success Manager, (Customer Experience...",The Purely Recruitment Company Ltd,Didcot,year,60000.00,60000.00,1,1,"customer success manager, customer experience...",the purely recruitment company ltd
4961,Employment Advisor - Fixed Term Contract (JETS...,Activate Learning,Oxford,year,27012.00,27012.00,0,0,employment advisor fixed term contract jets...,activate learning
4963,Guest Speaker\nnew,Oxford Summer Courses,Oxford,hour,100.00,153800.00,1,1,guest speaker,oxford summer courses
4964,Multi Skilled Operative\nnew,Grand Union Housing Group,Brackley,year,28773.00,28773.00,1,1,multi skilled operative,grand union housing group


In [30]:
df_to_clean.drop(columns=['Title', 'Company', 'salary_mean', 'Salary_freq'], axis=1, inplace=True)

In [31]:
df_to_clean.head()

,Location,salary_mean_normalised,High or low (50%),High or low (75%),Title clean,Company_clean
0,London,30000.0,1,1,computer vision and machine learning engineer,flox ltd
3,Stratford,31000.0,1,1,data and research analyst,cancer research uk
5,London,60000.0,1,1,data scientist verv london or sheffield,austin consultants
6,London,134400.0,1,1,data scientist machine learning,harnham
7,London,51500.0,1,1,principal data scientist,nesta


In [32]:
df_to_clean.to_csv('df_to_clean_v2.csv', index=False)